In [1]:
!pip install tensorflow-hub

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [3]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [4]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Add custom classification layers on top
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # For binary classification

In [6]:
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)


In [7]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [8]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy','Precision','Recall','AUC']
)

In [9]:
batch = 32
classes = 'binary'

train='C:/Users/Shusmita/Desktop/Oral Cancer/Augmented_Oral_Cancer/train'
test='C:/Users/Shusmita\Desktop\Oral Cancer/Augmented_Oral_Cancer/test'
val='C:/Users/Shusmita/Desktop/Oral Cancer/Augmented_Oral_Cancer/Validate'

In [10]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
validation_datagen= image.ImageDataGenerator()

test_datagen= image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)


validation_generator = validation_datagen.flow_from_directory(
    val,
    target_size = (224,224),
    batch_size = batch,
    shuffle=True,
    class_mode = classes)

test_generator = test_datagen.flow_from_directory(
    test,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)
class_names=validation_generator.class_indices
class_names
len(class_names)

Found 619 images belonging to 2 classes.
Found 130 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


2

In [11]:
# Train the model
epochs = 5  # You can adjust the number of epochs
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/5
20/20 [==============================] - 212s 11s/step - loss: 18.2261 - accuracy: 0.7092 - precision: 0.7079 - recall: 0.7170 - auc: 0.7191 - val_loss: 5.7456 - val_accuracy: 0.8308 - val_precision: 1.0000 - val_recall: 0.6562 - val_auc: 0.8414
Epoch 2/5
20/20 [==============================] - 232s 12s/step - loss: 2.7617 - accuracy: 0.8611 - precision: 0.8689 - recall: 0.8521 - auc: 0.8804 - val_loss: 1.8216 - val_accuracy: 0.8846 - val_precision: 1.0000 - val_recall: 0.7656 - val_auc: 0.9048
Epoch 3/5
20/20 [==============================] - 214s 11s/step - loss: 1.4864 - accuracy: 0.8982 - precision: 0.9161 - recall: 0.8778 - auc: 0.9192 - val_loss: 0.3927 - val_accuracy: 0.9462 - val_precision: 0.9524 - val_recall: 0.9375 - val_auc: 0.9580
Epoch 4/5
20/20 [==============================] - 205s 10s/step - loss: 0.4893 - accuracy: 0.9128 - precision: 0.8978 - recall: 0.9325 - auc: 0.9599 - val_loss: 0.3028 - val_accuracy: 0.9538 - val_precision: 0.9833 - val_recall: 0.92

In [12]:
score=model.evaluate(validation_generator)
score

5/5 [==============================] - 34s 7s/step - loss: 0.3018 - accuracy: 0.9154 - precision: 1.0000 - recall: 0.8281 - auc: 0.9902


[0.3018408417701721, 0.9153845906257629, 1.0, 0.828125, 0.9901752471923828]

In [13]:
score=model.evaluate(test_generator)
score

5/5 [==============================] - 35s 7s/step - loss: 0.4179 - accuracy: 0.9429 - precision: 0.9853 - recall: 0.9054 - auc: 0.9699


[0.4178904891014099,
 0.9428571462631226,
 0.9852941036224365,
 0.9054054021835327,
 0.9699017405509949]

In [14]:
score=model.evaluate(train_generator)
score

20/20 [==============================] - 169s 8s/step - loss: 0.1574 - accuracy: 0.9532 - precision: 0.9796 - recall: 0.9260 - auc: 0.9887


[0.1573716551065445,
 0.95315021276474,
 0.9795918464660645,
 0.9260450005531311,
 0.9887093901634216]